In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Convolution with Batch Norm & ReLU

In [ ]:
def ConvBNReLU(filters, kernel_size, x, strides=1,):
    if(kernel_size == 1):
        padding = 0
    else:
        padding = 1        
    # Conv2D
    x = ZeroPadding2D(padding)(x)
    x = Conv2D(filters, kernel_size, strides)(x)
    # Batch Normalization
    x = BatchNormalization()(x)
    # Activation Function
    x = Activation("relu")(x)
    return x

# Depthwise Separable Convolution

In [ ]:
def depthwise_separable(filters, kernel_size, x):
    #add padding
    x = ZeroPadding2D(padding=1)(x)
    # Depthwise Convolution 3x3
    x = SeparableConv2D(filters, kernel_size)(x)
    # Batch Normalization
    x = BatchNormalization()(x)
    # Activation Function
    x = Activation("relu")(x)
    return x

# Separable Squeeze-Expand Block

In [ ]:
def SSEModel(squeeze, expand, x):
    #squeeze
    pointwise1 = ConvBNReLU(squeeze, 1, x)
    #expand
    pointwise2 = ConvBNReLU(expand, 1, pointwise1)
    depthwise = depthwise_separable(expand, 3, pointwise1)

    x = tf.keras.layers.Concatenate()([pointwise2, depthwise])
    return x

# Slim Module

In [ ]:
def SlimModule(squeeze, expand, dws, filtersNo, x):
    #SSE block 1
    SSE1 = SSEModel(squeeze, expand, x)
    #skip connection
    SkipConnection = 0
    #Dense
    dns = Dense(filtersNo, activation='relu')(SSE1)
    
    SkipConnection = Add()([x, dns])
    #SSE block 2
    SSE2 = SSEModel(squeeze, expand, SkipConnection)

    #depthwise separable conv
    x = depthwise_separable(dws, 3, SSE2)
    return x

# SlimNet - CNN

In [ ]:
inp = Input(shape=(224, 224, 3))
x = Conv2D(96, (7, 7), (2, 2), padding='same')(inp)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#Slim module 1
x = SlimModule(16, 64, 48, 96, x)
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

#Slim module 2
x = SlimModule(32, 128, 96, 48, x)
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

#Slim module 3
x = Dropout(0.5)(x)
x = SlimModule(48, 192, 144, 96, x)
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

#Slim module 4
x = Dropout(0.5)(x)
x = SlimModule(64, 256, 192, 144, x)
x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

x = GlobalAveragePooling2D()(x)

x = Dense(1, activation='sigmoid')(x)

x = tf.keras.Model(inputs=inp, outputs = x)

print(x.summary())

# Check for Data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator()
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('../CelebA dataset/train', target_size=(96, 96), class_mode='binary')
val_it = datagen.flow_from_directory('../CelebA dataset/val', target_size=(96, 96), class_mode='binary')
#test_it = datagen.flow_from_directory('./faces', target_size=(128, 128), class_mode='categorical')
# confirm the iterator work
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [ ]:
import tensorflow.keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("./SlimModel/Checkppoint/SlimNet_Checkpoint.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystop = EarlyStopping(monitor = "val_loss",
                         min_delta = 0,
                         patience = 3,
                         verbose = 1,
                         restore_best_weights = True)

callbacks = [earlystop, checkpoint]

opt = tensorflow.keras.optimizers.Adam(lr=0.01e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

x.compile(loss = 'binary_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])

nb_train_samples = 49893
nb_validation_samples = 6236

epochs = 100
batch_size = 300


history = x.fit(
    train_it,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks=callbacks,
    validation_data = val_it,
    validation_steps = nb_validation_samples // batch_size)


x.save("./SlimModel/SlimNet_Model.h5")

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model("./SlimModel/SlimNet_Model.h5")

# Show the model architecture
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(val_it, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

# Test on a video

In [ ]:
import numpy as np
import cv2

from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
from os import listdir

def preprocess_image(image_path):
    """Loads image from path and resizes it"""
    img = load_img(image_path, target_size=(128, 128))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

def loadSlimFaceModel():
    """Loads the SlimNetFace model defined in the function"""
 
    inp = Input(shape=(128, 128, 3))
    x = Conv2D(96, (7, 7), (2, 2), padding='same')(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    #Slim module 1
    x = SlimModule(16, 64, 48, 96, x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    #Slim module 2
    x = SlimModule(32, 128, 96, 48, x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    #Slim module 3
    x = Dropout(0.5)(x)
    x = SlimModule(48, 192, 144, 96, x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    #Slim module 4
    x = Dropout(0.5)(x)
    x = SlimModule(64, 256, 192, 144, x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)

    x = GlobalAveragePooling2D()(x)

    
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)

    x = Dense(1, activation='sigmoid')(x)

    x = tf.keras.Model(inputs=inp, outputs = x)

    from keras.models import model_from_json
    x.load_weights('./SlimModel/Checkppoint/SlimNet_Checkpoint.h5')

    SlimNet_face_descriptor = Model(inputs=x.layers[0].input, outputs=x.layers[-2].output)

    return SlimNet_face_descriptor

model = loadSlimFaceModel()
print("Model Loaded")


# Test on a mp4 vide

In [ ]:
from os import listdir
from os.path import isfile, join
import cv2

# Loading HAARCascade Face Detector 
face_detector = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')

# Directory of image of persons we'll be extracting faces from
mypath = "./friends/"
image_file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print("Collected image names")

for image_name in image_file_names:
    person_image = cv2.imread(mypath+image_name)
    face_info = face_detector.detectMultiScale(person_image, 1.3, 5)
    for (x,y,w,h) in face_info:
        face = person_image[y:y+h, x:x+w]
        roi = cv2.resize(face, (128, 128), interpolation = cv2.INTER_CUBIC)
    path = "./friends_faces/" + "face_" + image_name 
    cv2.imwrite(path, roi)
    cv2.imshow("face", roi)
    
    cv2.waitKey(0)
cv2.destroyAllWindows()

# We load our faces from the "friends_faces" directory and we run our face classifier model our test video

In [ ]:
#points to your extracted faces
people_pictures = "./friends_faces/"

all_people_faces = dict()

for file in listdir(people_pictures):
    person_face, extension = file.split(".")
    all_people_faces[person_face] = model.predict(preprocess_image('./friends_faces/%s.jpg' % (person_face)))[0,:]

print("Face representations retrieved successfully")

def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

#Open Webcam or video
#cap = cv2.VideoCapture(0) 
cap = cv2.VideoCapture('testfriends.mp4')

while(True):
    ret, img = cap.read()
    img = cv2.resize(img, (640, 360)) # Re-size video to as smaller size to improve face detection speed
    faces = face_detector.detectMultiScale(img, 1.3, 5)

    for (x,y,w,h) in faces:
        if w > 13: 
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
            detected_face = cv2.resize(detected_face, (128, 128)) #resize to 224x224

            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 255

            captured_representation = model.predict(img_pixels)[0,:]

            found = 0
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]

                similarity = findCosineSimilarity(representation, captured_representation)
                if(similarity < 0.30):
                    cv2.putText(img, person_name[5:], (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                    found = 1
                    break

            #connect face and text
            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),(255, 0, 0),1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),(255, 0, 0),1)

            if(found == 0): #if found image is not in our people database
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('img',img)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

#kill open cv things
cap.release()
cv2.destroyAllWindows()